## Section 1: QKeras

In [16]:
# Switching for colab
%tensorflow_version 1.x

In [17]:
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, Adamax

from qkeras import QDense, QActivation, binary, quantized_bits

import os
import matplotlib.pyplot as plt
import numpy as np

In [5]:
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train / 255.0
    x_test = x_test / 255.0

    B_train = x_train.shape[0]
    B_test = x_test.shape[0]

    x_train = x_train.reshape(x_train.shape + (1,))
    x_test = x_test.reshape(x_test.shape + (1,))

    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    y_train = y_train + np.random.uniform(0.0, 0.02, y_train.shape)
    y_train = y_train / np.sum(y_train, axis=1, keepdims=True)
    
    x_train = x_train.reshape(x_train.shape[0], -1)
    x_test = x_test.reshape(x_test.shape[0], -1)
    
    return x_train, y_train, x_test, y_test

In [6]:
def get_model(verbose=True):
    x = x_i = Input((784,), name="input")
    x = Dense(300, name="d1")(x)
    x = Activation("relu", name="relu1")(x)
    x = Dense(100, name="d2")(x)
    x = Activation("relu", name="relu2")(x)
    x = Dense(10, name="d3")(x)
    x = Activation("softmax", name="softmax")(x)
    
    model = Model(inputs=x_i, outputs=x)

    if verbose:
        model.summary();
    
    return model

In [8]:
def get_model_q(alphas, verbose=True):
    x = x_i = Input((784,), name="input")
    x = QDense(300, name="d1",
               kernel_quantizer=binary(alpha=alphas.pop(0)),
               bias_quantizer=quantized_bits(4,2,1))(x)
    x = QActivation("binary()", name="act1")(x)
    x = QDense(100, name="d2",
               kernel_quantizer=binary(alpha=alphas.pop(0)),
               bias_quantizer=quantized_bits(4,2,1))(x)
    x = QActivation("binary()", name="act2")(x)
    x = QDense(10, name="d3",
               kernel_quantizer=binary(alpha=alphas.pop(0)),
               bias_quantizer=quantized_bits(4,2,1))(x)
    x = Activation("softmax", name="softmax")(x)
    
    model = Model(inputs=x_i, outputs=x)

    if verbose:
        model.summary();
    
    return model

In [9]:
def train(data, verbose=True):
    x_train, y_train, x_test, y_test = data
    weight_file_name = "elen521-lab6.hdf5"
    model = get_model(verbose=verbose)
    optimizer = Adam(0.01)

    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["acc"])
    
    if os.path.exists(weight_file_name):
        model.load_weights(weight_file_name)
    else:
        history = model.fit(
            x_train, y_train,
            batch_size=64,
            epochs=5,
            validation_split=0.05,
            verbose=True,
            shuffle=True)
        model.save_weights(weight_file_name)
    
    eval_res = model.evaluate(x_test, y_test)
    if verbose:
        print(eval_res)
    
    return eval_res, model

In [10]:
def get_alpha(model):
    layers_with_weights = ["d1", "d2", "d3"]
    alphas = []

    for layer in model.layers:
        if layer.name in layers_with_weights:
            weight_and_bias = layer.get_weights()
            weight = weight_and_bias[0]
            alphas.append(np.mean(np.abs(weight)))

    return alphas

In [11]:
def train_q(data, alphas, verbose=True):
    x_train, y_train, x_test, y_test = data
    model = get_model_q(alphas, verbose=verbose)
    optimizer = Adamax(0.02, beta_1=0.9, beta_2=0.999)

    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["acc"])

    history = model.fit(
        x_train, y_train,
        batch_size=64,
        epochs=5,
        validation_split=0.05,
        verbose=True,
        shuffle=True)
    
    eval_res = model.evaluate(x_test, y_test)
    if verbose:
        print(eval_res)
    
    return eval_res, model


In [12]:
def main():
    data = load_data()
    eval_res, model = train(data)
    
    alphas = get_alpha(model)
    eval_res_q = train_q(data, alphas)

In [13]:
main()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 784)]             0         
_________________________________________________________________
d1 (Dense)                   (None, 300)               235500    
_________________________________________________________________
relu1 (Activation)           (None, 300)               0         
_________________________________________________________________
d2 (Dense)                   (None, 100)               30100     
_________________________________________________________________
relu2 (Activation)           (None, 100)               0         
_________________________________________________________________
d3 (Dense)                   (None, 10)                1010      
______________________________________________

## Section 2 + 3: Neural Style Transfer

In [16]:
# Switching for colab
%tensorflow_version 2.x

In [ ]:
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, Adamax

import os
import matplotlib.pyplot as plt
import numpy as np